# Encoder-Decoderモデル

ニューラル機械翻訳用のEncoder-DecoderモデルをChainerで書いてみます。  
[公式のexample](https://github.com/chainer/chainer/blob/master/examples/seq2seq/seq2seq.py)を参考にしましたので、そちらも見てみてください。  

# モデルの記述

## ライブラリのimport

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

## ニューラル機械翻訳（NMT)のEncoder-Decoderモデル

NMTにおけるEncoder-Decodeモデルは、主に二つのコンポーネントを持っています。  

- Encoder: ソース言語の文を、ベクトルに変換する。
- Decoder: Encoderの出力をターゲット言語の文に変換する。

以下では、それぞれ分けて書いていきます。

まず、可変長のid系列のリストをembeddingの系列に変換する関数`sequence_embed`を用意しましょう。

In [2]:
# 可変長単語id系列を可変長単語ベクトル系列へ写像する関数
def sequence_embed(embed, xs):
    x_len = [len(x) for x in xs]
    x_section = np.cumsum(x_len[:-1])
    ex = embed(F.concat(xs, axis=0))
    exs = F.split_axis(ex, x_section, 0)
    return exs

### Encoder

Encoderモデルを書きます。  
入力単語系列をembeddingの系列に変換し、それらをNStepLSTMでencodeします。

In [3]:
class Encoder(Chain):
    def __init__(self, n_layers, n_source_vocab, n_units, dropout):
        super(Encoder, self).__init__()
        with self.init_scope():
            self.source_embed = L.EmbedID(in_size=n_source_vocab, out_size=n_units)
            self.encoder_lstm = L.NStepLSTM(n_layers=n_layers, in_size=n_units,
                                            out_size=n_units, dropout=dropout)
            self.n_source_vocab = n_source_vocab
            self.n_units = n_units
    
    def __call__(self, source_xs):
        # 単語の系列を単語ベクトルへ
        exs = sequence_embed(self.source_embed, source_xs)
        
        # lstmの初期状態
        hx = None
        cx = None
        
        # lstmで各系列をエンコード
        hy, cy, ys = self.encoder_lstm(hx, cx, exs)
        return hy, cy, ys
        

### Decoder

次にDecoderを書きます。  
DecoderもEmbedIDとNStepLSTMで書いて、あとでEncoder-Decodeモデルとしてまとめたクラスを書くことにします。

In [4]:
class Decoder(Chain):
    def __init__(self, n_layers, n_target_vocab, n_units, dropout):
        super(Decoder, self).__init__()
        with self.init_scope():
            self.target_embed = L.EmbedID(in_size=n_target_vocab, out_size=n_units)
            self.decoder_lstm = L.NStepLSTM(n_layers=n_layers, in_size=n_units,
                                            out_size=n_units, dropout=dropout)
            self.n_target_vocab = n_target_vocab
            self.n_units = n_units
        
    def __call__(self, hy, cy, target_xs):
        # targetの単語系列を単語ベクトルへ
        exs = sequence_embed(self.target_embed, target_xs)
        
        # encoderの出力を受け取り、lstmでデコード
        ho, co, os = self.decoder_lstm(hy, cy, exs)
        
        # 各タイムステップのhidden layerを返す
        return ho, co, os

### Encoder-Decoderモデル

EncoderとDecoderを書いたので、それらをつなげます。

NMTではEncoder-Decoderをいきなりひとつのモデルで定義しても問題ないと思いますが、今回はモデルを分けて保存・再利用できるようにするために、別々のクラスで書きました。

学習時のloss計算用のメソッドと予測時の翻訳用のメソッドを書くことにします。  


In [5]:
# UNKとEOSのidを指定しておく
UNK = 0
EOS = 1

In [6]:
class Encoder_Decoder(Chain):
    def __init__(self, encoder, decoder):
        super(Encoder_Decoder, self).__init__()
        with self.init_scope():
            # EncoderとDecoderを引数で受け取るようにしておく。
            self.encoder = encoder
            self.decoder = decoder
            
            # Decoderの隠れ状態からtargetボキャブラリーを予測する線形変換用のレイヤー
            self.W = L.Linear(self.decoder.n_target_vocab)
            # self.xp にはnumpyかcupyが勝手に入る
    
    def __call__(self, xs, ys):
        # xs: sourceの単語idの系列のリスト
        # ys: targetの単語idの系列のリスト
        
        # ソース言語を反転させて学習させるならここでそういった処理を入れましょう。
        
        # EOS=1
        # targetの単語sequenceにEOSをくっつける。
        # 学習時のDecoderの入力においては系列の前に(ys_in)
        # 答え合わせの時は系列の後に(ys_out)
        eos = self.xp.array([EOS], 'i')
        ys_in = [F.concat([eos, y], axis=0) for y in ys]
        ys_out = [F.concat([y, eos], axis=0) for y in ys]
        
        # encode & decode
        hy, cy, _ = self.encoder(xs)
        _, _, os = self.decoder(hy, cy, ys_in)
        
        # loss calculation
        batch_size = len(xs)
        concat_os = F.concat(os, axis=0)
        concat_ys_out = F.concat(ys_out, axis=0)
        loss = F.sum(F.softmax_cross_entropy(
            self.W(concat_os), concat_ys_out, reduce='no')) / batch_size
        
        return loss
    
    def translate(self, xs, max_length = 100):
        batch_size = len(xs)
        
        # 予測なので、backpropやdropoutをしないモードでモデルを動かす
        with chainer.no_backprop_mode(), chainer.using_config('train', False):
            # sourceの系列のエンコード
            h, c, _ = self.encoder(xs)
            
            # decode時のinput用にbatch_size分のEOS=1を用意
            ys = self.xp.full((batch_size,1), 1, 'i')
            result = []
            
            for i in range(max_length):
                h, c, ys = self.decoder(h, c, ys)
                cys = F.concat(ys, axis=0)
                wy = self.W(cys)
                ys = self.xp.argmax(wy.data, axis=1).astype('i')
                result.append(ys)
                ys = F.reshape(ys, (-1, 1)).data
            
            result = self.xp.stack(result).T
            # Remove EOS tags
            outs = []
            for y in result:
                inds = np.argwhere(y == EOS)
                if len(inds) > 0:
                    y = y[:inds[0, 0]]
                outs.append(y)
        return outs

---

# 実際に動かす

実際に上で記述したEncoder-Decoderモデルを動かしてみましょう。  

まずはデータを用意します。  
今回はごく小規模なデータにちゃんとフィッティングできるかを確かめるだけにします。  
[日英中基本文データ](http://nlp.ist.i.kyoto-u.ac.jp/index.php?日英中基本文データ)から１００個の日英対訳文をとってきて、日本語はMecabで分かち書き、英語は小文字化を行いました。  
それぞれ`ja.txt`、`en.txt`として、dataディレクトリに入れてあります。  
このデータを学習してみます。  

英日翻訳を試してみます。

## データ読み込み、前処理
まずはデータを見てみましょう。

In [28]:
with open('data/ja.txt','r') as f:
    ja = f.read().strip().split('\n')

with open('data/en.txt','r') as f:
    en = f.read().strip().split('\n')
    
ja = [s.strip() for s in ja]
en = [s.strip() for s in en]

In [31]:
for j, e in zip(ja[:10], en[:10]):
    print(j, e)

X で は ない か と つくづく 疑問 に 思う i often wonder if it might be x.
X が いい な と いつも 思い ます i always think x would be nice.
それ が ある よう に いつも 思い ます it always seems like it is there.
それ が 多 すぎ ない か と 正直 思う i honestly feel like there is too much.
山田 は みんな に 好か れる タイプ の 人 だ と 思う i think that yamada is the type everybody likes.
〜 と 誰 か が 思っ た someone thought that 〜
X は しんどい こと だ と 思い ます x seems like it's really tough.
X は 時間 の 問題 と 思い ます i think x is just a matter of time.
X は 今後 の 課題 と 思い ます i think that x will become an issue in the future.
それ は 桃山 時代 前後 の 作品 だ と 思い ます i think this was made around the momoyama period.


このような文のペアが１００個あります。

今回は単語を元に翻訳を行います。  
それぞれの単語をidに置き換えて、idの系列をモデルに入力していくことになります。  
よって、単語とidの辞書を作る必要があります。  
訓練データに出てくる単語にUNK(UNKNOWN)とEOS(End of Sentence)のトークンを付け加えた辞書を作りましょう。  
今回はUNKは0、EOSには1のidを振ります。 

※ 今回はvalidation/test dataでの性能評価は行わないので、実はUNKをつける意味はありません。

In [32]:
ja_w2id = {'UNK':0, 'EOS':1}
en_w2id = {'UNK':0, 'EOS':1}

In [34]:
ja_vocab = set()
for s in ja:
    ja_vocab.update(s.split(' '))
    
en_vocab = set()
for s in en:
    en_vocab.update(s.split(' '))

In [37]:
print(len(ja_vocab), len(en_vocab))

298 345


In [38]:
for i, w in enumerate(ja_vocab):
    ja_w2id[w] = i+2

In [41]:
for i, w in enumerate(en_vocab):
    en_w2id[w] = i+2

英語と日本語、それぞれの単語をidに置き換える辞書ができました。  
保存しておきます。

In [44]:
import pickle
with open('data/ja_w2id.dump', 'wb') as f:
    pickle.dump(ja_w2id, f)

with open('data/en_w2id.dump', 'wb') as f:
    pickle.dump(en_w2id, f)

それは文のリストを単語id系列のリストに変換します。  
numpy.arrayで単語id系列のリストにしていきましょう。

In [49]:
ja_data = []
for s in ja:
    s = s.split(' ')
    s = [ja_w2id[w] for w in s]
    ja_data.append(np.array(s, 'i'))

In [51]:
en_data = []
for s in en:
    s = s.split(' ')
    s = [en_w2id[w] for w in s]
    en_data.append(np.array(s, 'i'))

In [53]:
for i in zip(ja_data[:10], en_data[:10]):
    print(i)

(array([129, 160, 261,   7, 262,  16, 139, 144, 108, 163], dtype=int32), array([312, 253,   5, 292, 196, 251, 213, 270], dtype=int32))
(array([129,  39, 226,  15,  16,  44, 193,  30], dtype=int32), array([312, 346,  19, 113, 135, 213,  51], dtype=int32))
(array([238,  39,  13,  89, 108,  44, 193,  30], dtype=int32), array([196, 346, 290,  58, 196, 216,  23], dtype=int32))
(array([238,  39, 280, 272,   7, 262,  16,  63, 163], dtype=int32), array([312,  33,  47,  58,  35, 216, 223,  90], dtype=int32))
(array([ 23, 261,  93, 108,  76, 133, 206, 281,  79, 260,  16, 163], dtype=int32), array([312,  19,  24, 258, 216,  29, 162, 277, 311], dtype=int32))
(array([298,  16, 297, 262,  39,  17,  27], dtype=int32), array([108, 206,  24,  87], dtype=int32))
(array([129, 261,  22, 290, 260,  16, 193,  30], dtype=int32), array([113, 290,  58,  71, 180, 245], dtype=int32))
(array([129, 261, 215, 281,  37,  16, 193,  30], dtype=int32), array([312,  19, 113, 216, 282, 112, 302, 262, 332], dtype=int32))


これがモデルへの入力になります。

## モデル用意

モデルを準備します。  
今回は以下のようなモデルを学習しましょう。

- レイヤー数は1
- embedding, LSTMの中間層の次元は50。
- dropout rateは0.2
- 最適化はAdamで行う。

In [97]:
# 英語の文のEncoder
encoder = Encoder(n_layers=1, n_source_vocab=len(en_w2id), n_units=50, dropout=0.2)

# 日本語の文のDecoder
decoder = Decoder(n_layers=1, n_target_vocab=len(ja_w2id), n_units=50, dropout=0.2)

# Encoder-Decoderモデル。
model = Encoder_Decoder(encoder, decoder)

試しにちょっとだけデータを食わせてみます。

In [93]:
model(en_data[:2], ja_data[:2])

variable(56.64057540893555)

lossの値が返ってくるのがわかります。

最適化のためのoptimizerを定義しましょう。

In [98]:
optimizer = optimizers.Adam(0.01)
optimizer.setup(model)

これで、学習のための準備が整いました。

## 学習

20epochほど回してみて、5epochごとにどれくらいうまくフィッティングできているかを確かめましょう。  
ミニバッチサイズは10とします。  
今回は最初の3文を翻訳してみて、確かめてみます。 


In [99]:
for_translate_en = en_data[:3]

翻訳用の関数を用意しましょう。  
idから単語に戻すための辞書も用意しておきます。

In [100]:
ja_id2w = {i:w for w, i in ja_w2id.items()}
en_id2w = {i:w for w, i in en_w2id.items()}

In [101]:
def translate_print(en_array, ja_array, en_id2w, ja_id2w):
    print('=====中間報告=====')
    for en, ja in zip(en_array, ja_array):
        en_s = [en_id2w[i] for i in en]
        print(' '.join(en_s))
        
        ja_s = [ja_id2w[i] for i in ja]
        print('>>> '+' '.join(ja_s))
    print('\n')

In [102]:
translate_print(en_data[:3], ja_data[:3], en_id2w, ja_id2w)

=====中間報告=====
i often wonder if it might be x.
>>> X で は ない か と つくづく 疑問 に 思う
i always think x would be nice.
>>> X が いい な と いつも 思い ます
it always seems like it is there.
>>> それ が ある よう に いつも 思い ます




動いてます。

それでは学習を開始してみましょう。

In [104]:
# 訓練データ総数
n_train = len(ja_data)

# ミニバッチサイズ
batchsize = 10

# エポック数
n_epoch =20

# ミニバッチ化のために、array化しておく。dtypeはobject
ja_data = np.array(ja_data)
en_data = np.array(en_data)

for epoch in range(n_epoch): # epochのループ
    sum_loss = 0
    # ミニバッチの用意。
    perm = np.random.permutation(n_train)
    for i in range(0, n_train, batchsize):
        ja_batch = ja_data[perm[i:i+batchsize]]
        en_batch = en_data[perm[i:i+batchsize]]
        
        # lossの計算
        loss = model(en_batch, ja_batch)
        sum_loss += loss.data
        # backpropatgation
        optimizer.target.cleargrads() # 勾配のリセット
        loss.backward() # 逆伝搬する誤差を計算
        optimizer.update() # パラメータを更新
    
    log = '# epoch: {}, loss: {}'.format(epoch+1, sum_loss)
    print(log)
    
    # 5epochごとに翻訳を出力。
    if (epoch+1)%5 == 0:
        result = model.translate(for_translate_en)
        translate_print(for_translate_en, result, en_id2w, ja_id2w)
        
    
    

# epoch: 0, loss: 522.3270835876465
# epoch: 1, loss: 367.9393253326416
# epoch: 2, loss: 301.5761775970459
# epoch: 3, loss: 254.57226181030273
# epoch: 4, loss: 211.6100616455078
=====中間報告=====
i often wonder if it might be x.
>>> X は は の と 思い ます
i always think x would be nice.
>>> X が X を し た
it always seems like it is there.
>>> それ が の を する


# epoch: 5, loss: 172.45455741882324
# epoch: 6, loss: 136.8543004989624
# epoch: 7, loss: 105.37115383148193
# epoch: 8, loss: 79.51851654052734
# epoch: 9, loss: 59.56546878814697
=====中間報告=====
i often wonder if it might be x.
>>> X で は ない か と つくづく 疑問 に 思う
i always think x would be nice.
>>> X が いい な と いつも 思い ます
it always seems like it is there.
>>> それ が ある よう に いつも 思い ます


# epoch: 10, loss: 44.27755522727966
# epoch: 11, loss: 33.51788568496704
# epoch: 12, loss: 25.33184826374054
# epoch: 13, loss: 19.647087335586548
# epoch: 14, loss: 15.441965579986572
=====中間報告=====
i often wonder if it might be x.
>>> X で は ない か と つくづく 疑問 に 思う
i alwa

最初の5epochではダメな感じですが、訓練データに含まれている文なので、さすがに次の5epochではフィッティングできていますね。  
lossもちゃんと減っていっています。

今回は学習にあまり時間をかけないようにするために、小さな訓練データを用いてモデルを訓練し、訓練データの文を使ってフィッティングを検証していますが、  

実際の機械学習では、ちゃんとtrain/val/testにデータを分割しなければなりません。    
単語id辞書作成や学習は訓練データのみで行い、valデータでモデル選択、テストデータでモデルの評価を行うようにしてください。  

---

# まとめ

お疲れ様でした。  
ChainerでEncoder-Decoderモデルを定義し、小規模データを使った機械翻訳の学習を行いました。  
次は今回やったことの発展として、Attentionを導入します。